# AWS SSM Paramter Store
- AWS에서 주요 설정과 값들을 저장/관리/활용하기 위한 서비스
    - 예 : API 주소, DB 호스트명, AMI ID, API Token, 유저아이디.패스워드, 환경변수 등
- Key Value 기반으로 필요한 값을 저장하고 불러오기 가능
    - 예 : /myproject/prod/db/userid -> "my_user_id"
    - 저장 가능한 값의 형식
        - String : 텍스트
        - StringList : 컴마로 구분된 값 (예: Monday, Wednesday, Friday)
        - SecureString : KMS 기반으로 암호화된 텍스트
- IAM으로 권한 관리 가능
    - 접근 권한 (계층 구조 적용 가능)
    - 복호화 권한(SecureString)
- 다양한 AWS 서비스에서 활용
    - 예 : Lambda, EC2, CloudFormation 등

## AWS SSM Parameter Store 티어
- 최대 파라미터 숫자(리전 당)
    - 스탠다드 : 10000 / 어드밴스드 : 100000
- 최대 파라미터 사이즈
    - 스탠다드 : 4KB / 어드밴스드 : 8KB
- 파라미터 정책
    - 스탠다드 : X / 어드밴스드 : Y
- 비용
    - 스탠다드 : 무료 / 어드밴스드 : 유료
- 기타
    - 스탠다드 : 어드밴스드로 변경 가능 / 어드밴스드 : 스탠다드로 변경 불가능

## AWS SSM Parameter Store - SecureString
- KMS를 활용해서 파라미터를 암호화하여 저장
    - KMS Managed Key를 선택해 암호화(기본 aws/ssm 키)
    - 조회 시 WithDecryption 옵션을 넣어주어야 복호화된 값을 받을 수 있음
- 조회 권한과 Decrypt 권한이 분리 : 조회하려면 KMS 권한도 필요

## AWS SSM Parameter Store - Versioning
- Versioning 기능
    - 기본 1부터 내용 변경시마다 버전이 계속 증가
    - 최대 100개
        - 100개 이상 생성 시 예전 버전부터 삭제 (예외: Parameter Label)
- 참조시 parameter_name:version 형식으로 사용 가능
    - 예 : :/myproject/prod/ec2/ami_id:3
- 조회 시 특정 버전의 파라미터 조회 가능

## AWS SSM Paramter Store - Labels
- 유저가 각 버전에 부여하는 텍스트
    - 태그처럼 파라미터의 버전에 식별 가능한 목적/정보등을 부여 (최대 100글자)
    - 한 버전에 여러 Label 부여 가능(최대 10개)
    - 한 Label을 여러 버전에 부여할 수는 없음 : 중복 부여 불가능
- Label의 관리
    - 파라미터에 Label 추가/삭제 가능
    - 하나의 Label을 다른 파라미터로 이동 가능
    - 히스토리 조회 가능
- Label 단위로 조회 가능

## AWS SSM Parameter Store - 계층구조
- 파라미터 스토어의 키를 계층 구조로 관리 가능
    - 슬래쉬를 기반으로 각 계층을 구분
    - 첫 글자가 슬래쉬로 시작하지 않을 경우, 계층 구조 없는 파라미터로 생성 가능
- 예
- myproject
    - /prod
        - /db
            - /hostname : /myproject/prod/db/host : "121.11.25.11"
            - /user_id
        - /s3
            - /bucketname
    - /dev
        - /db
            - /hostname
        - /s3
            - /bucketname
- 활용
    - 계층 구조 단위로 조회 가능(GetParameterByPath)
        - 예: aws ssm get-parameters-by-path --path /myproject/prod/db = /hostname, /user_id를 가져옴
    - 계층 구조 단위로 권한 부여 가능
        - allow only "arn:aws:ssm:us-east-2:2:1234567:parameter/myproejct/dev"

## AWS SSM Parameter Store - Parameter Policy
- 파라미터에 지정할 수 있는 관리 정책 세가지
    - Expiration : 일정 기간 이후 삭제(TTL)
    - ExpirationNotification : 삭제 전에 Eventbridge 생성
    - NoChangeNotification : 일정 기간 이상 파라미터 변경이 없을 경우 EventBridge 이벤트 생성
    - Advanced Tier만 사용 가능
- 삭제/갱신 가능

## AWS SSM Parameter Stroe - Public Parameter
- AWS에서 공식적으로 배포하는 파라미터
    - 각 OS별 AMI의 아이디
    - AWS의 모든 리전 목록
    - AWS의 모든 서비스 목록
    - 특정 서비스를 사용 가능한 리전 목록
- 별도의 권한요구사항 없음